In [1]:
%%writefile task1_create_external_table.hql

add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

use demodb;

drop table if exists posts_sample_external;

create external table posts_sample_external (
    id int,
    post_type_id int,
    creation_date string,
    tags string,
    owner_user_id int,
    parent_id int,
    score int,
    favorite_count int
)
comment 'posts external'
row format serde 'org.apache.hadoop.hive.serde2.RegexSerDe'
    with serdeproperties (
        "input.regex" = '^\<row(?=.*\\bId=\"(\\d+))(?=.*\\bPostTypeId=\"(\\d+))(?=.*\\bCreationDate=\"([0-9-T:.]+)\")(?=.*\\bTags=\"([\\S\\s]+)\")?(?=.*\bOwnerUserId=\"(\\d+))?(?=.*\\bParentId=\"(\\d+))?(?=.*\\bScore=\"(\\d\+))?(?=.*\\bFavoriteCount =\"(\\d+))?.*$'
    )
stored as textfile
location '/data/stackexchange1000/posts'
tblproperties ('skip.header.line.count' = '1');

Overwriting task1_create_external_table.hql


In [2]:
%%writefile task1_check_select.hql

add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

use demodb;

select * from posts_sample_external limit 10;

Overwriting task1_check_select.hql


In [3]:
%%writefile task1_create_managed_table.hql

add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

use demodb;

drop table if exists posts_sample;

create table posts_sample (
    id int,
    post_type_id int,
    creation_date string,
    tags string,
    owner_user_id int,
    parent_id int,
    score int,
    favorite_count int
)
comment 'posts managed'
partitioned by (year int, month string)
row format delimited
    fields terminated by '\t'
    lines terminated by '\n'
stored as textfile;

Overwriting task1_create_managed_table.hql


In [4]:
%%writefile task1_insert_table.hql

add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

use demodb;

set hive.exec.dynamic.partition.mode=nonstrict;

from posts_sample_external
insert overwrite table posts_sample
partition (year, month)
select 
    id,
    post_type_id,
    creation_date,
    tags,
    owner_user_id,
    parent_id,
    score,
    favorite_count,
    split(creation_date, '-')[0],
    split(creation_date, '-')[1];

Overwriting task1_insert_table.hql


In [5]:
%%writefile task1_watch_new_table.hql

add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

use demodb;

describe posts_sample;

Overwriting task1_watch_new_table.hql


In [6]:
%%writefile task1_result.hql

add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

use demodb;

select concat_ws('\t', string(year), concat(year, '-', month), string(cnt)) 
    from (
        select year, month, count(1) as cnt, row_number() over(order by year, month) as rnum 
            from posts_sample 
            group by year, month
    ) posts 
    where rnum = 3;

Overwriting task1_result.hql


In [7]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

In [8]:
!cat task1.hql


add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

use demodb;

drop table if exists posts_sample_external;

create external table posts_sample_external (
    id int,
    post_type_id int,
    creation_date string,
    tags string,
    owner_user_id int,
    parent_id int,
    score int,
    favorite_count int
)
comment 'posts external'
row format serde 'org.apache.hadoop.hive.serde2.RegexSerDe'
    with serdeproperties (
        "input.regex" = '^\<row(?=.*\\bId=\"(\\d+))(?=.*\\bPostTypeId=\"(\\d+))(?=.*\\bCreationDate=\"([0-9-T:.]+)\")(?=.*\\bTags=\"([\\S\\s]+)\")?(?=.*\bOwnerUserId=\"(\\d+))?(?=.*\\bParentId=\"(\\d+))?(?=.*\\bScore=\"(\\d\+))?(?=.*\\bFavoriteCount =\"(\\d+))?.*$'
    )
stored as textfile
location '/data/stackexchange1000/posts'
tblproperties ('skip.header.line.count' = '1');
add jar /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

use demodb;

drop table if exists posts_sample;



In [9]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

<IPython.core.display.Javascript object>

In [10]:
%%bash
hive -f task1.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
FAILED: SemanticException [Error 10072]: Database does not exist: demodb
